In [17]:
import tensorflow as tf
import numpy as np
import json
import os

# Daftar path model .keras dan path untuk menyimpan model .json
keras_model_paths = ['all_models.h5']
json_model_paths = ['all_models.json']

# Fungsi untuk mengkonversi model Keras ke JSON
def convert_keras_to_json(keras_model_path, json_model_path):
    try:
        # Muat model dari file .keras
        model = tf.keras.models.load_model(keras_model_path)
        
        # Konversi arsitektur model ke JSON
        model_json = model.to_json()

        # Ekstrak bobot model
        weights = model.get_weights()

        # Konversi bobot model ke dalam format yang dapat disimpan ke JSON
        weights_json = [w.tolist() for w in weights]
        
        # Gabungkan arsitektur dan bobot ke dalam satu dictionary
        model_data = {
            'model_architecture': model_json,
            'model_weights': weights_json
        }
        
        # Simpan model ke file JSON
        with open(json_model_path, 'w') as json_file:
            json.dump(model_data, json_file)
        print(f"Model {keras_model_path} telah berhasil dikonversi dan disimpan di {json_model_path}")
    except Exception as e:
        print(f"Error saat mengkonversi {keras_model_path}: {str(e)}")

# Path untuk model JSON
json_model_path = 'all_models.json'

# Konversi model .h5 ke .json
convert_keras_to_json('model/best_model_low.keras', json_model_path)

# Konversi semua model dalam daftar
for keras_path, json_path in zip(keras_model_paths, json_model_paths):
    convert_keras_to_json(keras_path, json_path)


Error saat mengkonversi model/best_model_low.keras: File not found: filepath=model/best_model_low.keras. Please ensure the file is an accessible `.keras` zip file.
Model all_models.h5 telah berhasil dikonversi dan disimpan di all_models.json


In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import json
import tensorflow as tf

# Function to load model architecture and weights from JSON
def load_model_from_json(json_model_path):
    with open(json_model_path, 'r') as json_file:
        model_data = json.load(json_file)

    # Parse model architecture and weights from JSON data
    model_architecture = model_data['model_architecture']
    model_weights = np.array(model_data['model_weights'])  # Convert weights to numpy array

    return model_architecture, model_weights

# Function to rebuild model from architecture and weights
def rebuild_model(model_architecture, model_weights):
    # Create a new model from the model architecture
    model = tf.keras.models.model_from_json(model_architecture)

    # Set the model weights
    model.set_weights(model_weights)

    return model

# Function to make prediction for next month
def predict_next_month_sales(data, model, window_size=12):
    data['date'] = pd.to_datetime(data['date'])
    monthly_sales = data.resample('M', on='date').sum()
    
    # Extract sales data for the previous 11 months
    previous_11_months_sales = monthly_sales['sales'].iloc[-window_size-1:-1].values
    latest_month_sales = monthly_sales['sales'].iloc[-1]

    # Prepare input data for prediction
    input_data = np.array(previous_11_months_sales).reshape((1, window_size, 1))

    # Scale the input data
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(input_data.reshape(-1, 1))
    input_data_scaled = scaler.transform(input_data.reshape(-1, 1)).reshape((1, window_size, 1))

    # Make prediction
    predicted_sales_scaled = model.predict(input_data_scaled)
    
    # Inverse transform to get actual sales value
    predicted_sales_actual = scaler.inverse_transform(predicted_sales_scaled.reshape(-1, 1))

    return predicted_sales_actual[0][0]

# Path to JSON model file
json_model_path = 'all_models.json'

# 1. Load model from JSON file
model_architecture, model_weights = load_model_from_json(json_model_path)

# 2. Rebuild model
model = rebuild_model(model_architecture, model_weights)

# Load new data from CSV file
new_data_url = 'https://raw.githubusercontent.com/C4AnN/Models/final/dataset/file_gabungan2.csv'
new_data = pd.read_csv(new_data_url)

# Ensure 'date' column is in datetime format
new_data['date'] = pd.to_datetime(new_data['date'])

# Apply categorization
new_data['category'] = new_data['price'].apply(categorize_price)

# Filter data for each category
new_data_low = new_data[new_data['category'] == 'low'].copy()
new_data_mid = new_data[new_data['category'] == 'mid'].copy()
new_data_high = new_data[new_data['category'] == 'high'].copy()

# Predict sales for next month for each category
predicted_sales_low = predict_next_month_sales(new_data_low, model)
predicted_sales_mid = predict_next_month_sales(new_data_mid, model)
predicted_sales_high = predict_next_month_sales(new_data_high, model)

# Output predictions
print("Prediksi penjualan untuk bulan berikutnya (Low Category):", predicted_sales_low, "units")
print("Prediksi penjualan untuk bulan berikutnya (Mid Category):", predicted_sales_mid, "units")
print("Prediksi penjualan untuk bulan berikutnya (High Category):", predicted_sales_high, "units")


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8,) + inhomogeneous part.